In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
from sklearn.linear_model import LinearRegression
import imblearn
from imblearn import over_sampling
from matplotlib import *
from scipy.stats import ttest_ind
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import *
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from numpy.random import choice
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from scipy.stats import bootstrap
from imblearn.metrics import specificity_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from typing import Tuple
import copy as cp
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn import preprocessing

In [ ]:
##Cargar datos: Excel con Features y Respuesta Patólogica: 2: Respuesta No Completa -1: Respuesta Completa##

file="Features_Completos.xlsx"
df=pd.read_excel(file)
df
df.rename(columns={"Diagnosis (0:Respuesta No Completa-1:Respuesta Completa)":"Diagnosis"},inplace=True)
df=df.dropna()

df.head()

In [ ]:
##Se realiza el split de los datos: Train/Test (70-30%)#



y=df["Respuesta Patologica (1: Respuesta Completa; 0:Respuesta No Completa)"].values


X_train,X_test,y_train, y_test=train_test_split(df,y,test_size=0.3,random_state=1)
feature_names=X_train.columns[1:390].values

#Se normalizan los datos #

stand= MinMaxScaler()
stand.fit(X_train)

X_train_norm = stand.transform(X_train)
X_test_norm= stand.transform(X_test)

X_train_N = pd.DataFrame(X_train_norm, columns=df.columns)
X_test_N =  pd.DataFrame(X_test_norm, columns=df.columns)

In [ ]:
#Test Mann-Whitney con alpha=0.05
from scipy.stats import ttest_ind
from scipy.stats import wilcoxon
from scipy.stats import ranksums
from IPython.display import Markdown
import math
feature_signif=[]
alpha=0.05
T_hip=[]
p_hip=[]
diag=y_train.reshape(-1,1)

for name in feature_names:
    feature=X_train_N[name].values.reshape(-1,1)
   
    feature_RC=[]
    feature_RNC=[]
    for i in range(diag.size):
        if diag[i]==1:
            feature_RC.append(feature[i])
            
        elif diag[i]==0:
            feature_RNC.append(feature[i])

    feature_RC=np.asarray(feature_RC)
    feature_RNC=np.asarray(feature_RNC)


    ttest,pval=ranksums(feature_RC,feature_RNC)

    T_hip.append(ttest)
    p_hip.append(pval)
    
 
   



    if pval<alpha:
            Markdown('<span style="color: #008000">Se rechaza H0</span>')
            fRC=np.asarray(feature_RC)
            fRNC=np.asarray(feature_RNC)
           
            feature_signif.append(name)
    else:
            Markdown('<span style="color: #ff0000">No es posible rechazar H0</span>')

In [ ]:
#Evaluación de features con diferencia significativa
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import seaborn as sns
modelos=[]
modelos.append(("LR",LogisticRegression(solver="liblinear",multi_class="ovr")))
modelos.append(("KNN",KNeighborsClassifier()))
modelos.append(("DT",DecisionTreeClassifier()))
modelos.append(("NB",GaussianNB()))
modelos.append(("SVM",SVC(kernel= "linear",gamma="auto",probability=True)))


best_model=[]#mejor modelo para cada feature
best_value=[]#valor del promedio de la metrica para el mejor modelo



from numpy.random import choice 
for feature in feature_signif:
    results=[]
   
    


  
    y=y_train
    X=np.asarray(X_train_N[feature])
   
    clf2 = LogisticRegression(solver="liblinear").fit(X.reshape(-1,1), y)
    cv_results=roc_auc_score(y, clf2.predict_proba(X.reshape(-1,1))[:, 1])
    results.append(cv_results)
        
       
        
        
    values=np.mean(results)
 

 
    best_model.append("LR")
    best_value.append((values))

  

    
    feature_AUC=results 

In [ ]:
#Analisis de correlación - se selecciona los features con AUC>0.7
def getKey(item):
    return item[2]
F = [f for f,m,v in sorted(zip(feature_signif,best_model,best_value),key=getKey,reverse=True)]
M = [m for f,m,v in sorted(zip(feature_signif,best_model,best_value),key=getKey,reverse=True)]
V = [v for f,m,v in sorted(zip(feature_signif,best_model,best_value),key=getKey,reverse=True)]


corr=X_train_N[X_train_N[F].columns].corr()
list_features=corr.columns.values
f=F[:]

for names in F:
    R=[f for f in abs(corr[names])>0.8]
    lista=list_features[R].tolist()
    lista.remove(names)
    for i in range(len(lista)):
        if lista[i] in F:
            F.remove(lista[i])

m=[]
v_0=[]
v=[]
#rr=np.ones((len(F),k*repeated))
FF=[]
AUC=[]
Y=[]
for i in range(len(F)):
    m.append(M[f.index(F[i])])
    
    v_0.append(V[f.index(F[i])])
    if v_0[i]>=0.7:
            v.append(v_0[i])
          

In [ ]:
#Modelo Multivariado
from itertools import combinations
R=[]
M =[]
S=[]
CI=[]

features_used=[]
Val=[]


k=5
repeated=100
Rep=0

n_feat=[]

for i in range(2,10):
    c=np.asarray(list(set(combinations(FF,i))))
  
    n_feat.append(i)
    for j in range(len(c)):
        results = []
        names = []
            
        X=np.asarray(X_train_N.loc[:,c[j]])
        X_t=np.asarray(X_test_N.loc[:,c[j]])
        for name,model in modelos:
                    model=model
                    selector=RFE(model, n_features_to_select=i, step=1)
                    kfold=StratifiedKFold(n_splits=k, random_state=None)
                    kRfold=RepeatedStratifiedKFold(n_splits=k, n_repeats=repeated, random_state=0)
                    pipeline=Pipeline(steps=[("sc",selector),("m",model)])
                    cv_results=cross_val_score(pipeline,X,y,cv=kRfold,scoring="roc_auc")
                    results.append(cv_results)
                    names.append(name)

        values=[np.mean(results[i]) for i in range(4)]
        clf=modelos[values.index(max(values))][1]

        clf.fit(X,y)

        cv_results2=roc_auc_score(y_test,clf.predict_proba(X_t)[:,1])
        Val.append(cv_results2)
        R.append(modelos[values.index(max(values))][0])
        M.append(modelos[values.index(max(values))][1])
        S.append(max(values))
        features_used.append(c[j])
        indice=values.index(max(values))
        
        feature_AUC=results[indice]
        
        conf_intervalAUC=np.percentile(feature_AUC,[5,95])
        
        CI.append(conf_intervalAUC)



In [ ]:
indice= S.index((max(S)))
number_feat1=int(len(features_used[indice]))
print("El numero optimo de features es:",number_feat1)
print(*features_used[indice], sep="\n")
print(R[indice])
print(max(S))
features_seleccionados= features_used[indice]

In [ ]:
#Validación con Base externa#
file_val="Val_Comp.xlsx"
df_val=pd.read_excel(file_val)
df_val.rename(columns={"Diagnosis (0:Respuesta No Completa-1:Respuesta Completa)":"Diagnosis"},inplace=True)
df_val=df_val.dropna()

df_val.head()

features_seleccionados= features_used[indice]


clf1=M[indice]
X=np.asarray(X_train_N.loc[:,features_seleccionados])

clf1.fit(X,y)
X_norm_val = stand.transform(df_val)
X_norm_val= pd.DataFrame(X_norm_val, columns=df.columns)
X_val=np.asarray(X_norm_val.loc[:,features_seleccionados])


#X_val=np.asarray(X_norm_val.loc[:,features_seleccionados])
y_val=df_val["Respuesta Patologica (1: Respuesta Completa; 0:Respuesta No Completa)"].values
#X_val=np.asarray(X_norm_val.loc[:,features_seleccionados])

auc=roc_auc_score(y_val,clf1.predict(X_val))
report_val=classification_report(y_val,clf1.predict(X_val))
print(auc)
print(report_val)

In [ ]:
# Entrenamos con la base externa y realizamos el test con la interna:

In [ ]:
clf1.fit(X_val,y_val)
auc2=roc_auc_score(y,clf1.predict(X))
print("El auc:", auc2)